In [1]:
data_var = '2024-01-04'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8718,2024-01-04,Eua Nba,00:00,Los Angeles Lakers,Miami Heat,1.50,2.88,227.5,1.86,2.07,-5.5,1.97,1.97,zkedL8Ac,0.666667,0.347222,0.537634,0.483092,0.013889,0.4,0.4,NaN,NaN,263.014,71.119022,0.270400,298.898,80.409607,0.269020,195.646,299.704,151.62,400.40,0.0,0.0,0.0,0.0,0.445574,0.075569,0.000000,-2.31,-0.462,-1.082251,0.643869,0.6,-0.043869,4.62,0.924,2.034632,0.453011,0.6,0.146989
8719,2024-01-04,Eua Nba,00:00,Sacramento Kings,Orlando Magic,1.54,2.73,229.5,1.80,2.08,-4.5,1.93,2.01,hQk1KSP3,0.649351,0.366300,0.555556,0.480769,0.015651,0.4,0.2,NaN,NaN,197.852,42.514841,0.214882,223.210,63.741873,0.285569,174.738,307.328,165.62,321.00,0.0,0.0,0.0,0.0,0.394125,0.102057,0.028715,-1.01,-0.202,-2.673267,0.623411,0.7,0.076589,-1.58,-0.316,-5.474684,0.415289,0.4,-0.015289
8720,2024-01-04,Eua Nba,21:30,San Antonio Spurs,Milwaukee Bucks,5.11,1.22,248.5,1.80,1.87,8.5,2.05,1.71,WGl5JnuA,0.195695,0.819672,0.555556,0.534759,0.015367,0.8,0.8,NaN,NaN,427.778,176.779196,0.413250,202.970,34.490528,0.169929,387.100,200.258,530.25,179.69,1.0,0.0,0.0,0.0,0.869082,0.026974,0.127881,-1.65,-0.330,-12.454545,0.329362,0.1,-0.229362,-0.02,-0.004,-55.000000,0.617087,0.6,-0.017087
8721,2024-01-04,Europa Euroliga,14:45,Fenerbahce,Maccabi Tel Aviv,1.51,2.64,169.5,1.91,1.91,-5.5,2.00,1.71,2eaiI7Eg,0.662252,0.378788,0.523560,0.523560,0.041040,0.8,0.6,NaN,NaN,147.704,64.842600,0.439004,168.106,38.596459,0.229596,146.420,199.052,99.56,207.06,0.0,0.0,0.0,0.0,0.385075,0.000000,0.110545,2.15,0.430,1.186047,0.000000,0.0,0.000000,3.13,0.626,2.619808,0.000000,0.0,0.000000
8722,2024-01-04,Europa Euroliga,16:00,Crvena zvezda,Partizan,1.76,2.14,164.5,1.91,1.91,-3.5,2.10,1.67,KW9nJmam,0.568182,0.467290,0.523560,0.523560,0.035472,0.0,0.4,NaN,NaN,184.702,100.335767,0.543231,139.190,91.059453,0.654210,108.614,156.596,81.60,100.94,0.0,0.0,0.0,0.0,0.137795,0.000000,0.161303,-0.75,-0.150,-5.066667,0.757041,0.8,0.042959,-0.88,-0.176,-6.477273,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8813,2024-01-04,Itália Coppa Italia,15:30,Ragusa F,Faenza F,1.06,7.60,139.5,1.85,1.85,-15.5,1.85,1.85,KWuRRsRj,0.943396,0.131579,0.540541,0.540541,0.074975,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.068009,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8814,2024-01-04,Itália Coppa Italia,16:00,San Martino F,Oxygen Roma F,1.30,3.31,143.5,1.85,1.85,-7.5,1.85,1.85,ttsNSNBp,0.769231,0.302115,0.540541,0.540541,0.071346,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.616609,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8815,2024-01-04,Itália Coppa Italia,16:00,Sassari F,Brixia F,1.22,3.98,142.5,1.85,1.85,-9.5,1.85,1.85,0nafX33T,0.819672,0.251256,0.540541,0.540541,0.070928,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.750621,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8816,2024-01-04,Itália Coppa Italia,16:30,San Giovanni F,Sanga Milano F,1.08,7.01,131.5,1.85,1.85,-15.5,1.85,1.85,YZT8jR3i,0.925926,0.142653,0.540541,0.540541,0.068579,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Lakers,Miami Heat,227.5,1.86,1.0,Over
1,00:00,Eua Nba,Sacramento Kings,Orlando Magic,229.5,1.80,1.0,Over
